# Python TFM Section

## Modelling
1. Compare several models and use the most suitable for this case of study
2. Train and Test the final model and set it up implemented
3. Final conclusions

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from pathlib import Path

In [9]:
df_modelize = pd.read_csv(Path.cwd() / 'Outputs' / 'DF_MODELIZE.csv', sep = ';', header = 0 , encoding = "ISO-8859-1",decimal=',')
df_modelize.sample(5)

,VERSION,ID_UPR,ID_TECNOLOGIA,A. No Cobrados,AMONIACO,ATR,Ajuste,BONO_SOCIAL_PEGO,Banda,Bilateral,...,Res. Pot. Adicional,S. Regulacion,S. Res. Pot. Adicional,SERV_GEST_RES,TASAS_MEDIOAMB,TASA_ARAGON,Terciaria,UREA,VCF,POWER_MWH
862,201808,UPR300,GN,NaN,NaN,-159848.0,NaN,NaN,1635496.0,NaN,...,17682718.0,NaN,-7778.0,NaN,0.0,NaN,1786978.0,NaN,12638.0,1224801.602
971,201811,UPR1860,CI,NaN,NaN,NaN,NaN,NaN,9629500.0,50815237.0,...,NaN,NaN,-1468.0,NaN,-31877.0,NaN,4033062.0,-13027.0,257152.0,9606446.850
642,201803,UPR2414,BP,NaN,NaN,NaN,-7655317.0,NaN,NaN,-4030849.0,...,NaN,NaN,-434.0,NaN,NaN,NaN,-437218.0,NaN,NaN,-877594.240
840,201808,UPR1752,BP,NaN,NaN,NaN,-45090782.0,NaN,NaN,NaN,...,NaN,NaN,-10259.0,NaN,NaN,NaN,5000255.0,NaN,NaN,244890.280
948,201810,UPR303,HN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-6642.0,NaN,0.0,NaN,NaN,NaN,NaN,-103614.300
